In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
from pandas_plink import read_plink
import matplotlib.pyplot as plt

In [ ]:
pheno_id=14
grad_clean=np.loadtxt("/ourdisk/hpc/nullspace/adbadre/auto_archive_notyet/tape_2copies/result_mean.csv")

In [ ]:
bim=pd.read_csv("/work/biobank/ukb_data/new_data/all.bim",sep="\t",header=None)

In [ ]:
chro=list(bim[0].values)+[27 for i in range(805426)]

In [ ]:
grad_clean_loc=grad_clean[:,pheno_id]

In [ ]:
a=pd.DataFrame(np.array([chro,grad_clean_loc]).T,columns=["CHR","P"])
a=a.astype({'CHR': 'int32'})

In [ ]:
th="0.001"
print(pd.read_csv("/ourdisk/hpc/nullspace/adbadre/auto_archive_notyet/tape_2copies/results_final_first_order_other_method.csv",index_col=0)[th])

In [ ]:
'''manhattan plot'''
df=a
df['ind'] = range(len(df))
df_grouped = df.groupby(('CHR'))
fig,ax=plt.subplots(figsize=(50,20))
colors = ['dimgrey', 'grey']
x_labels = []
x_labels_pos = []
for num, (name, group) in enumerate(df_grouped):
    '''Remove sex chromosomes and plot the rest of it'''
    if name not in (23,24,25,26):
        #print(name,group)
        if name==27:
            group.plot(kind='scatter', x='ind', y='P',color="orange", ax=ax,fontsize=20,alpha=0.3)
            x_labels_pos.append((group['ind'].iloc[-1] - (group['ind'].iloc[-1] - group['ind'].iloc[0])/2))
        else:
            group.plot(kind='scatter', x='ind', y='P',color=colors[num % len(colors)], ax=ax,fontsize=20)
            x_labels_pos.append((group['ind'].iloc[-1] - (group['ind'].iloc[-1] - group['ind'].iloc[0])/2))
        x_labels.append(name)
        
x_labels[-1]=" "
ax.set_xticks(x_labels_pos)
ax.set_xticklabels(x_labels)
ax.set_xlim([0, len(df)])
ax.axhline(np.sort(grad_clean_cerv)[::-1][65],color="purple",label="0.1%",linestyle="dashed",linewidth=3)
ax.axhline(np.sort(grad_clean_cerv)[::-1][64924],color="blue",label="5%",linestyle="dashed",linewidth=3)
ax.set_xlabel(' ')
ax.set_ylabel(' ')
plt.xticks(fontsize=25)
plt.yticks(fontsize=25)
plt.ticklabel_format(style='sci', axis='y',)
plt.legend(fontsize=30)
ax.set_ylim([-0.0001, 0.007])

In [ ]:
'''Remove sex chromosomes'''
one=grad_clean_cerv[:784256]
two=grad_clean_cerv[805426:]
tot=np.append(one,two)

In [ ]:
'''make categories'''
l=["Real SNPs" if x<784256 else "Decoy SNPs" for x in range(len(tot))]

In [ ]:
d=pd.DataFrame(np.array([tot.T.astype(float),l]).T)

In [ ]:
d[0]=d[0].astype(float)

In [ ]:
d

In [ ]:
fig,ax=plt.subplots(figsize=(20,10))
d.groupby(1)[0].plot(kind="density",legend=True,color = {'Real SNPs': 'grey', 'Decoy SNPs': 'orange'})
ax.set_ylabel("")
ax.set_xlim([-0.0001, 0.007])
ax.invert_xaxis()
plt.tick_params(
    axis='both',          # changes apply to the x-axis
    labelsize=0, length = 0)
plt.legend(fontsize=30)

In [ ]:
'''Code to calculate the intersection sizes at 0.1%'''

imp=pd.read_csv("/ourdisk/hpc/nullspace/adbadre/auto_archive_notyet/tape_2copies/results_final_first_order_other_method.csv",index_col=0)["0.001"]

In [ ]:
imp

In [ ]:
c=0
snps=[]
for i in imp:
    print(i)
    if np.isnan(i):
        i=0
    snps.append(np.argsort(grad_clean[:,c])[::-1][:int(i)])
    c+=1

In [ ]:
imp

In [ ]:
phenos=[x for x in range(17)]#[0,2,3,4,5,12,13,14,15,16]
a=np.zeros((len(phenos),len(phenos)))
d=imp.index.values[[phenos]]
for i in range(len(phenos)):
    for j in range(i+1,len(phenos)):
        inter_1d_loc=np.intersect1d(snps[phenos[i]],snps[phenos[j]])
        inter_1d_loc=inter_1d_loc[inter_1d_loc<784256]
        a[i,j]=len(inter_1d_loc)
        a[j,i]=len(inter_1d_loc)
        print(d[i],d[j],np.intersect1d(snps[phenos[i]],snps[phenos[j]]))

In [ ]:
a[np.isnan(a)]=0
a

In [ ]:
df_a=pd.DataFrame(a,index=d,columns=d)

In [ ]:
l=["malignant melanoma","non-melanoma skin cancer","skin cancer","lung cancer","intrathoracic cancer","colorectal cancer","colon cancer","rectal cancer","bladder cancer","uterine cancer","cervical cancer","prostate cancer","breast cancer","female genital tract cancer","male genital tract cancer","lymphoma","non-hodgkins lymphoma"]

i=['lung cancer', 'breast cancer', 'skin cancer',
       'colorectal cancer', 'colon cancer',
       'rectal cancer', 'bladder cancer', 'female genital tract cancer',
       'male genital tract cancer', 'uterine cancer',
       'cervical cancer', 'prostate cancer', 'lymphoma',
       'non-hodgkins lymphoma', 'malignant melanoma',
       'non-melanoma skin cancer', 'intrathoracic cancer']

In [ ]:
df_a.index=i
df_a.columns=i
df_a=df_a.loc[l,l]

In [ ]:
df_a

In [ ]:
df_a.to_csv("/ourdisk/hpc/nullspace/adbadre/auto_archive_notyet/tape_2copies/intersect1d_tot_01_nums.csv")

In [ ]:
'''Code to plot Venn Diagrams'''
th="0.001"
imp=pd.read_csv("/ourdisk/hpc/nullspace/adbadre/auto_archive_notyet/tape_2copies/results_final_first_order_other_method.csv",index_col=0)[th]

c=0
snps=[]
for i in imp:
    #print(i)
    if np.isnan(i):
        i=0
    snps.append(np.argsort(grad_clean[:,c])[::-1][:int(i)])
    c+=1

snpi=snps[phenos[14]]
snpi=snpi[snpi<784256]
print(imp.index.values[14],len(snpi))
snpj=snps[phenos[9]]
snpj=snpj[snpj<784256]
print(imp.index.values[9],len(snpj))

snpk=snps[phenos[3]]
snpk=snpk[snpk<784256]
print(imp.index.values[3],len(snpk))

def plotVenn(snpi,snpj,snpk,namei,namej,namek):
    dlpv=len(np.intersect1d(snpi,np.intersect1d(snpj,snpk)))
    dl=len(np.intersect1d(snpi,snpj))-dlpv
    interdl=np.intersect1d(snpi,snpj)
    print(dlpv)
    dpv=len(np.intersect1d(snpk,snpi))-dlpv
    lpv=len(np.intersect1d(snpk,snpj))-dlpv
    nd=len(snpi)-dlpv-dl-dpv
    nl=len(snpj)-dlpv-lpv-dl
    npv=len(snpk)-dlpv-lpv-dpv
    plt.figure(figsize=(10,10))
    v = venn3(subsets=(nl, nd, dl, npv, lpv, dpv, dlpv), set_labels = (namej,namei, namek))#(nl, nd, dl, npv, lpv, dpv, dlpv)
    v.get_label_by_id('A').set_text('')
    v.get_label_by_id('B').set_text('')
    v.get_label_by_id('C').set_text('')
    for text in v.subset_labels:
        try:
            text.set_fontsize(15)
        except:
            pass
         arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0.5',color='gray'))
    plt.show()

plotVenn(snpi,snpj,snpk,imp.index.values[14],imp.index.values[9],imp.index.values[3])

In [ ]:
'''Code to calculate union correlation'''
imp=pd.read_csv("/ourdisk/hpc/nullspace/adbadre/auto_archive_notyet/tape_2copies/results_final_first_order_other_method.csv",index_col=0)["0.05"]

In [ ]:
c=0
snps=[]
for i in imp:
    print(i)
    if np.isnan(i):
        i=0
    snps.append(np.argsort(grad_clean[:,c])[::-1][:int(i)])
    c+=1

In [ ]:
import scipy.stats

In [ ]:
'''Calculate the union correlations'''
phenos=[i for i in range(17)]
a=np.zeros((len(phenos),len(phenos)))
d=imp.index.values[[phenos]]
inter_sex={}
for i in range(len(phenos)):
    for j in range(i+1,len(phenos)):
        SNPsi=snps[phenos[i]]
        SNPsj=snps[phenos[j]]
        union_loc=np.union1d(SNPsi,SNPsj)
        union_loc=union_loc[union_loc<784256]
        loci=grad_clean[:,i][union_loc]
        locj=grad_clean[:,j][union_loc]
        a[i,j]=scipy.stats.spearmanr(loci,locj)[0]
        a[j,i]=scipy.stats.spearmanr(loci,locj)[0]   
df_a=pd.DataFrame(a,index=d,columns=d)

In [ ]:
df_a.columns

In [ ]:
l=["malignant melanoma","non-melanoma skin cancer","skin cancer","lung cancer","intrathoracic cancer","colorectal cancer","colon cancer","rectal cancer","bladder cancer","uterine cancer","cervical cancer","prostate cancer","breast cancer","female genital tract cancer","male genital tract cancer","lymphoma","non-hodgkins lymphoma"]

i=['lung cancer', 'breast cancer', 'skin cancer',
       'colorectal cancer', 'colon cancer',
       'rectal cancer', 'bladder cancer', 'female genital tract cancer',
       'male genital tract cancer', 'uterine cancer',
       'cervical cancer', 'prostate cancer', 'lymphoma',
       'non-hodgkins lymphoma', 'malignant melanoma',
       'non-melanoma skin cancer', 'intrathoracic cancer']

In [ ]:
df_a.index=i
df_a.columns=i
df_a=df_a.loc[l,l]

In [ ]:
df_a[df_a>=0.5].count().sum()

In [ ]:
df_a[df_a<0].count().sum()

In [ ]:
intermediate=df_a[df_a>=0]
intermediate[intermediate<0.5].count().sum()

In [ ]:
df_a.to_csv("/ourdisk/hpc/nullspace/adbadre/auto_archive_notyet/tape_2copies/union1d_5.csv")